In [1]:
! git clone -b master https://github.com/charles9n/bert-sklearn
! cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 10.12 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 132 kB 15.1 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 8.7 MB 44.6 MB/s 
     |████████████████████████████████| 138 kB 55.3 MB/s 
     |████████████████████████████████| 127 kB 16.8 MB/s 
  Created wheel for bert-sklearn: filena

In [2]:
! pip install bert_sklearn

In [3]:
! pip install awscli --ignore-installed six

     |████████████████████████████████| 3.8 MB 27.6 MB/s 
     |████████████████████████████████| 547 kB 61.0 MB/s 
  Using cached botocore-1.25.9-py3-none-any.whl (8.7 MB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
     |████████████████████████████████| 636 kB 32.9 MB/s 
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
     |████████████████████████████████| 247 kB 79.4 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datascienc

In [4]:
import torch
import codecs
import pandas as pd
import numpy as np
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
class TSV(object):
    """
    .tsv file's handler.
    """
 
    def __init__(self, file):
        """
        TSV init.
        :param file: .tsv file to handle.
        """
        self.file = file
 
    def __repr__(self):
        return "File {file} under handling......".format(file=self.file)
 
    def tsv(self):
        """
        .tsv file's column definition and data check.
        :return: List.
            lines data from [file] row by row in dict format.
        """
        with codecs.open(self.file, 'r', "utf-8") as f:
            line = f.readline()
            data = []
            head = []
            while line:
                if line.isspace():
                    line = f.readline()
                    continue
                elif not line.isspace():
                    # to be compatible between OS
                    head = line.rstrip("\r\n").split('\t')
                    line = f.readline()
                    break
            while line:
                if line.isspace():
                    line = f.readline()
                    continue
                elif not line.isspace():
                    body = line.rstrip("\r\n").split('\t')
                    rows = zip(head, body)
                    tsv_dic = {}
                    for (head_sub, body_sub) in list(rows):
                        tsv_dic[head_sub] = body_sub
                    data.append(tsv_dic)
                    line = f.readline()
            return data
 
 
if __name__ == "__main__":
    with codecs.open("tsv", 'w', "utf-8") as f:
        rows = """
        #Id\tContent
        #1\tContent1
        #2\tContent2
        #3\tContent3
        #4\tContent4
        #1024\tContent1024
        #"""
        f.writelines(rows.replace(' ', ''))
    TSV_Tester = TSV(file="/content/drive/MyDrive/Colab Notebooks/BERT NER/Semana_2_Marzo/Extracción de Síntomas con CRF/Expedientes Medicos.tsv")

In [7]:
Exp_Med = TSV_Tester.tsv()

data = pd.DataFrame(Exp_Med)
data1 = np.array(Exp_Med)

lista_Palabra = list(data['"Nota'])
lista_Etiqueta = list(data['O'])

ls_tupla = []
for i in range(len(lista_Palabra)):
    ls_tupla.append([lista_Palabra[i], lista_Etiqueta[i]])
ls_tupla.insert(0, ['"Nota', 'O'])

In [8]:
list_Notas_1 = []
ls_1 = []
for i in ls_tupla:
   
    if '"Nota' not in i:
        ls_1.append(i)
    if '"Nota' in i:
        
        ls_1.insert(0, ['"Nota', 'O'])
        list_Notas_1.append(ls_1)
        ls_1.append(['.', 'O'])
        ls_1=[]
list_Notas_1.pop([0][0])



[['"Nota', 'O'], ['.', 'O']]

In [9]:
len(list_Notas_1)
list_Notas_2 = []
list_Notas_2 = list_Notas_1
for i in list_Notas_2:
    for j in i:
        if "Sintomas" not in j:
            j[1] = 'O'


In [10]:
# Realizamos una modificación para tener un formato BIO
for i in range(len(list_Notas_2)):
    c = 0
    #d = 0

    for j in range(len(list_Notas_2[i])):
        #print(list_Notas_2[i][j][1])


        if list_Notas_2[i][j][1] == 'Sintomas' and list_Notas_2[i][j-1][1] == 'O':
            #print('Hola')
            list_Notas_2[i][j][1] = 'B-SIN'
            c =+1

        if list_Notas_2[i][j][1] == 'Sintomas' and list_Notas_2[i][j-1][1] == 'B-SIN':
            #print('yy')
            list_Notas_2[i][j][1] = 'I-SIN'

        if list_Notas_2[i][j][1] == 'Sintomas' and list_Notas_2[i][j-1][1] == 'I-SIN':
            #print('yy')
            list_Notas_2[i][j][1] = 'I-SIN'

In [ ]:
#for i in list_Notas_2:
 #   c = 0
  #  for j in i:
   #     if c == 0:
    #        if j[1] == 'Sintomas':
     #           j[1] = 'B-SIN'
      #          c =+1
       # if c != 0:
        #    if j[1] == 'Sintomas':
         #       j[1] = 'I-SIN'

In [11]:
list_ = list_Notas_2
list_Notas_11 = []
ls_11 = []
ls_2 = []
for i in list_:
    for j in i:
        if '.' not in j[0]:
            ls_11.append(j)
        if '.' in j[0]:
            #ls_1.insert(0, ['"Nota', 'O'])
            ls_11.append(j)
            list_Notas_11.append(ls_11)
            ls_11=[]
    ls_2.append(list_Notas_11)
    list_Notas_11 = []
#list_Notas_1.pop([0][0])

In [12]:
Lista_Ora_Sin = []

for i in ls_2:
  for j in i:
    c = 0
    for k in j: 
      if 'O' not in k:
        if c == 0:
          Lista_Ora_Sin.append(j)
          c = c + 1

In [13]:
# Para las oraciones
Lista_Tokens = []
Lista_Lables = []


for i in Lista_Ora_Sin:
    l1_token = []
    l1_label = []
    for j in i:
        l1_token.append(j[0])
        l1_label.append(j[1])
    Lista_Tokens.append(l1_token)
    Lista_Lables.append(l1_label)



Lis_Etx = Lista_Tokens


test_ratio = 0.1

train_ratio = int((1.0-test_ratio)*len(Lis_Etx))


Lis_Et_trainx = Lis_Etx[0:train_ratio]
Lis_Et_testx = Lis_Etx[train_ratio:-1]

Lis_Ety = Lista_Lables


test_ratio = 0.1

train_ratio = int((1.0-test_ratio)*len(Lis_Ety))


Lis_Et_trainy = Lis_Ety[0:train_ratio]
Lis_Et_testy = Lis_Ety[train_ratio:-1]

df_train = pd.DataFrame(list(zip(Lis_Et_trainx,Lis_Et_trainy)), columns = ['tokens','labels'])
df_test = pd.DataFrame(list(zip(Lis_Et_testx,Lis_Et_testy)), columns=['tokens', 'labels'])

In [14]:
df_train[:15]

,tokens,labels
0,"[Al, pase, de, visita, se, refiere, asintomát...","[O, O, O, O, O, O, B-SIN, O, O, O, O, O, O, O,..."
1,"[Al, pase, de, visita, refiere, tos, seca, ,, ...","[O, O, O, O, O, B-SIN, I-SIN, O, O, O, O, O, O]"
2,"[Al, pase, de, visita, se, refiere, asintomát...","[O, O, O, O, O, O, B-SIN, O, O, O, O, O, O, O, O]"
3,"[Al, momento, del, pase, el, paciente, refiere...","[O, O, O, O, O, O, O, B-SIN, O, O, O]"
4,"[Al, pase, de, visita, se, refiere, con, disne...","[O, O, O, O, O, O, O, B-SIN, I-SIN, I-SIN, I-S..."
5,"[Al, momento, del, pase, el, paciente, refiere...","[O, O, O, O, O, O, O, B-SIN, I-SIN, I-SIN, I-S..."
6,"[Al, pase, de, visita, se, encuentra, alerta,,...","[O, O, O, O, O, O, O, O, O, B-SIN, O, O, O, O,..."
7,"[Al, momento, del, pase, el, paciente, refiere...","[O, O, O, O, O, O, O, B-SIN, O, O, B-SIN, O]"
8,"[Al, pase, de, visita,, se, refiere, asintomá...","[O, O, O, O, O, O, B-SIN, O, O, O, O, O, O, O,..."
9,"[Únicamente, refiere, pirosis, de, predominio...","[O, O, B-SIN, O, O, O, O, O, O, O, O, O, O, O,..."


In [15]:
X_train, y_train = df_train.tokens, df_train.labels
X_test, y_test = df_test.tokens, df_test.labels


y_trainn=[]
for i in y_train:
  for j in i:
    y_trainn.append(j)

    
label_list = np.unique(y_trainn)
label_list = list(label_list)
print("\nNER tags:",label_list)

# take a subset of the data for demo
n = 3861
#X_train, y_train = X_train[:n], y_train[:n]
#X_test, y_test = X_test[:n], y_test[:n]


NER tags: ['B-SIN', 'I-SIN', 'O']


In [16]:
%%time
# define model
model = BertTokenClassifier(bert_model='bert-base-multilingual-cased',
                            epochs = 10,
                            max_seq_length=201,
                            learning_rate=2e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            gradient_accumulation_steps=2,
                            validation_fraction=0.1,                            
                            label_list=label_list)#ignore_label=['O'])


print(model)

Building sklearn token classifier...
BertTokenClassifier(bert_model='bert-base-multilingual-cased', epochs=10,
                    eval_batch_size=16, gradient_accumulation_steps=2,
                    label_list=['B-SIN', 'I-SIN', 'O'], max_seq_length=201,
                    train_batch_size=16)
CPU times: user 4.29 ms, sys: 19 µs, total: 4.31 ms
Wall time: 4.04 ms


In [17]:
# finetune model
model.fit(X_train, y_train)

# score model
f1_test = model.score(X_test, y_test)
print("Test f1: %0.02f"%(f1_test))

# make predictions
y_preds = model.predict(X_test)

# calculate the probability of each class
y_probs = model.predict_proba(X_test)

#print(classification_report(flatten(y_test), flatten(y_preds)))

100%|██████████| 995526/995526 [00:00<00:00, 26512871.74B/s]


Loading bert-base-multilingual-cased model...


100%|██████████| 625/625 [00:00<00:00, 649192.67B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 64, validation data size: 7



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

Test f1: 93.86


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
y_preds = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
y_test1 = []
y_preds1 = []
for i in y_test:
    for j in i:
        y_test1.append(j)
for i in y_preds:
    for j in i:
        y_preds1.append(j)




In [20]:
! pip install sklearn_crfsuite

from sklearn_crfsuite import metrics

metrics.flat_f1_score(y_test, y_preds,
                      average='weighted', labels=label_list)


     |████████████████████████████████| 965 kB 37.1 MB/s 


0.9467779010779962

In [21]:
y_test2, y_preds2 = [], []
for i in range(len(y_test1)):
  if y_test1[i] != 'O':
    y_test2.append(y_test1[i])
    y_preds2.append(y_preds1[i])


from sklearn.metrics import classification_report
print(classification_report(y_test2, y_preds2))

              precision    recall  f1-score   support

       B-SIN       0.91      0.83      0.87        12
       I-SIN       1.00      0.60      0.75         5
           O       0.00      0.00      0.00         0

    accuracy                           0.76        17
   macro avg       0.64      0.48      0.54        17
weighted avg       0.94      0.76      0.83        17



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
! pip install seqeval



     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=17c35bc58c70341b629e3537e107be109608b3f99bd697e5d8ab20bc54a4304e
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [23]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         SIN       0.47      0.58      0.52        12

   micro avg       0.47      0.58      0.52        12
   macro avg       0.47      0.58      0.52        12
weighted avg       0.47      0.58      0.52        12



In [24]:
# Procesamos para ver las predicciones que está haciendo 

Palabra = []
for i in X_test:
    for j in i:
        Palabra.append(j)

Test = []
for i in y_test:
    for j in i:
        Test.append(j)

Predicciones = []
for i in y_preds:
    for j in i:
        Predicciones.append(j)

data_frame1 = pd.DataFrame(list(zip(Palabra, Test, Predicciones)), columns = ['Palabra','Test','Predicción'])


In [25]:
Lista_pred_BERT = list(zip(Palabra, Test, Predicciones))

In [26]:
Lista_oraciones_predicciones = []
for i in Lista_pred_BERT:
  if 'B-SIN' == i[1] or 'I-SIN' == i[1]:
    Lista_oraciones_predicciones.append(i)

In [27]:
dataframe_de_pred = pd.DataFrame(Lista_oraciones_predicciones, columns=['Palabra','Test','Predicción'])
dataframe_de_pred

,Palabra,Test,Predicción
0,Niega,B-SIN,B-SIN
1,Afebril,B-SIN,B-SIN
2,Negó,B-SIN,B-SIN
3,sintomatología,I-SIN,I-SIN
4,insuficiencia,B-SIN,O
5,respiratoria,I-SIN,O
6,odinofagia,B-SIN,B-SIN
7,tos,B-SIN,B-SIN
8,seca,I-SIN,B-SIN
9,cefalea,B-SIN,B-SIN


In [28]:
# Se importan las librerías

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import glob
import csv
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import numpy as np
from sklearn.metrics import classification_report
import json
import spacy
import random
import re
import codecs
import spacy

In [29]:
with open('/content/drive/MyDrive/Colab Notebooks/BERT NER/Semana_2_Marzo/Extracción de Síntomas con CRF/HG_Cabrera_samih_covid_10_08_2020.json') as file:
    data = json.load(file)
Lista_notas_Cabrera = []
ls = []
for i in data:
    for j in data[i]['nota']:
    #ls.append(('Síntomas:'+ data[i]['nota']['Síntomas']).split())
        ls.append((str(j) +':' + data[i]['nota'][j]).split())

    Lista_notas_Cabrera.append(ls)

In [30]:
data['1']

{'alcaldia': '',
 'chn': 526836,
 'edad': '53',
 'episodio': 'Episodio de Hospitalizacion',
 'estado': '',
 'fecha_alta': '',
 'fecha_hora_registro_nota': '10/09/2020 09:39:39                                      ',
 'fecha_ingreso': '31/08/2020 11:56:38',
 'fuente': '/Users/DanielO/Desktop/SS/proc_script/notas_medicas\\HG_Cabrera_samih_covid_10_08_2020.csv',
 'name': 'MARILU',
 'nota': {'Análisis': '\n\n\nPaciente que cursa su 11° día de EIH, día 15 desde el inicio \nde síntomas que se encuentra con puntas de alto flujo 25 litros por minuto y Fi\no2 de 30 por lo que se espera en las próximas horas progresar a puntas simples,\n ultima gasometría con alcalosis metabolica, Se mantiene estrecha vigilancia.  \nSe realiza cambio de puntas de alto flujo a Puntas simples con 5 litros/minuto s\naturando entre 95 y 97%\n',
  'Diagnóstico': '\n\n\n\n\nNeumonía por SARS COV 2+DMT2+Sobrepeso\n\n\n\n',
  'Objetivo': '\n\n\nFemenina de edad aparente igual a la cronológica, alerta, orien\ntada y coop

In [31]:
with open('/content/drive/MyDrive/Colab Notebooks/BERT NER/Semana_2_Marzo/Extracción de Síntomas con CRF/HG_Cabrera_samih_covid_10_08_2020.json') as file:
    data = json.load(file)

Lista_notas_Cabrera = []
for i in data:
    ls = []
    for j in data[i]['nota']:
        ls.append((str(j) +':' + data[i]['nota'][j]).split())

    Lista_notas_Cabrera.append(ls)


In [32]:
Notas_Cabrera = []
for i in Lista_notas_Cabrera:
  for j in i:
    Notas_Cabrera.append(j)

In [33]:
len(Notas_Cabrera)

2628

In [34]:
y_preds = model.predict(Notas_Cabrera)

/content/bert-sklearn/bert_sklearn/utils.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/165 [00:00<?, ?it/s]

In [35]:
# Procesamos para ver las predicciones que está haciendo 

Palabra = []
for i in Notas_Cabrera[:1000]:
    for j in i:
        Palabra.append(j)

Predicciones = []
for i in y_preds:
    for j in i:
        Predicciones.append(j)

data_frame = pd.DataFrame(list(zip(Palabra, Predicciones)), columns = ['Palabra','Predicción'])

In [36]:
data_frame

,Palabra,Predicción
0,Signos,O
1,Vitales:,O
2,10/09/2020,O
3,06:49:,O
4,Temperatura:,O
...,...,...
60963,Atención,O
60964,prioritaria,O
60965,especializada,O
60966,Signos,O


In [37]:
Lista_Predicciones = list(zip(Palabra, Predicciones))

In [38]:
Lista_oraciones_predicciones = []
for i in Lista_Predicciones:
  if 'B-SIN' == i[1] or 'I-SIN' == i[1]:
    Lista_oraciones_predicciones.append(i)

In [39]:
print('Se hicieron', len(Lista_oraciones_predicciones), 'predicciones')

Se hicieron 1491 predicciones


In [40]:
data_frame_pred = pd.DataFrame(Lista_oraciones_predicciones, columns = ['Palabra', 'Predicción'])

In [41]:
data_frame_pred[0:60]

,Palabra,Predicción
0,cardi,I-SIN
1,aca,I-SIN
2,Glicemia,B-SIN
3,síntomas,B-SIN
4,disnea,B-SIN
5,orien,B-SIN
6,coloración,I-SIN
7,de,I-SIN
8,tegumentos,I-SIN
9,hidratación,B-SIN


In [42]:
data_frame_pred[60:120]

,Palabra,Predicción
60,cardi,I-SIN
61,aca,I-SIN
62,Gastritis,B-SIN
63,dolor,I-SIN
64,torácico.,I-SIN
65,la,I-SIN
66,presencia,I-SIN
67,de,I-SIN
68,disnea.,B-SIN
69,de,I-SIN


# Análisis de 1 sola nota nueva y sus etiquetas

In [43]:
for i in data['20']['nota']:
    print(str(i) + ':' + data['20']['nota'][i])

Signos Vitales:


23/09/2020 04:39: Temperatura: 36.5      /      Frecuencia cardi
aca - ADL: 84.0      /      Frecuencia respiratoria - ADL: 18.0      /      SaO2
: 96.0      /                                                                   
Otras constantes de hoy:Tensión Arterial Sistólica - ADL: 114.0      /      Te
nsión Arterial Diastólica - ADL: 66.0      /      Tensión Arterial Media - AD
L: 82.0      /      Glicemia capilar: 88.0      /

Síntomas:


Se trata Arellano Ojeda Marilú  de 52 años de edad, con antece
dentes de Diabetes Mellitus tipo 2 + Hipertrigliceridemia + Obesidad grado II. C
on antecedente de estancia hospitalaria en Terapia Respiratoria Intermedia (C7):
 del día 05.09.20 al 11.09.20. Con antecedente de infección pulmonar sobreagre
gada (En tratamiento con linezolid). Con PCR (+) Al pase de visita la paciente n
iega dolor torácico. Negó sintomatología urinaria o digestiva. La paciente ni
ega la presencia de disnea.

Objetivo:


Mujer de edad aparente igual a 

In [44]:
lista_nota_1 = []
for i in data['20']['nota']:
    lista_nota_1.append((str(i) + ':' + data['20']['nota'][i]).split())

In [45]:
lista_nota_1

[['Signos',
  'Vitales:',
  '23/09/2020',
  '04:39:',
  'Temperatura:',
  '36.5',
  '/',
  'Frecuencia',
  'cardi',
  'aca',
  '-',
  'ADL:',
  '84.0',
  '/',
  'Frecuencia',
  'respiratoria',
  '-',
  'ADL:',
  '18.0',
  '/',
  'SaO2',
  ':',
  '96.0',
  '/',
  'Otras',
  'constantes',
  'de',
  'hoy:Tensión',
  'Arterial',
  'Sistólica',
  '-',
  'ADL:',
  '114.0',
  '/',
  'Te',
  'nsión',
  'Arterial',
  'Diastólica',
  '-',
  'ADL:',
  '66.0',
  '/',
  'Tensión',
  'Arterial',
  'Media',
  '-',
  'AD',
  'L:',
  '82.0',
  '/',
  'Glicemia',
  'capilar:',
  '88.0',
  '/'],
 ['Síntomas:',
  'Se',
  'trata',
  'Arellano',
  'Ojeda',
  'Marilú',
  'de',
  '52',
  'años',
  'de',
  'edad,',
  'con',
  'antece',
  'dentes',
  'de',
  'Diabetes',
  'Mellitus',
  'tipo',
  '2',
  '+',
  'Hipertrigliceridemia',
  '+',
  'Obesidad',
  'grado',
  'II.',
  'C',
  'on',
  'antecedente',
  'de',
  'estancia',
  'hospitalaria',
  'en',
  'Terapia',
  'Respiratoria',
  'Intermedia',
  '(C7):',
  

In [46]:
Predic_nota_1 = model.predict(lista_nota_1)

/content/bert-sklearn/bert_sklearn/utils.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
# Procesamos para ver las predicciones que está haciendo 

Palabra_nota_1 = []
for i in lista_nota_1:
    for j in i:
        Palabra_nota_1.append(j)

Predicciones_nota_1 = []
for i in Predic_nota_1:
    for j in i:
        Predicciones_nota_1.append(j)

data_frame_nota_1 = pd.DataFrame(list(zip(Palabra_nota_1, Predicciones_nota_1)), columns = ['Palabra de la nota 1','Predicción de la nota 1'])

In [48]:
Lista_Prediccionesn1 = list(zip(Palabra_nota_1, Predicciones_nota_1))

In [49]:
Lista_oraciones_prediccionesn1 = []
contador1 = 0
for i in Lista_Prediccionesn1:
  if 'B-SIN' == i[1] or 'I-SIN' == i[1]:
    Lista_oraciones_prediccionesn1.append([i, contador1])
  contador1 += 1


In [50]:
Lista_oraciones_prediccionesn1

[[('cardi', 'I-SIN'), 8],
 [('aca', 'I-SIN'), 9],
 [('Glicemia', 'B-SIN'), 50],
 [('Hipertrigliceridemia', 'B-SIN'), 74],
 [('dolor', 'I-SIN'), 117],
 [('torácico.', 'I-SIN'), 118],
 [('disnea.', 'B-SIN'), 131],
 [('síntomas.', 'B-SIN'), 272],
 [('oxigeno', 'I-SIN'), 288],
 [('glicemia', 'B-SIN'), 335],
 [('Neumonía', 'B-SIN'), 541],
 [('Diabetes', 'B-SIN'), 547],
 [('Mel', 'I-SIN'), 548],
 [('litus', 'I-SIN'), 549],
 [('ras', 'I-SIN'), 573]]

In [51]:
data_frame_nota_1

,Palabra de la nota 1,Predicción de la nota 1
0,Signos,O
1,Vitales:,O
2,23/09/2020,O
3,04:39:,O
4,Temperatura:,O
...,...,...
734,31/08/2020,O
735,16:14NEWS:,O
736,Atención,O
737,general,O


# Sólo síntomas de la nota 20

In [52]:
print('Síntomas' + ':' + data['20']['nota']['Síntomas'])

Síntomas:


Se trata Arellano Ojeda Marilú  de 52 años de edad, con antece
dentes de Diabetes Mellitus tipo 2 + Hipertrigliceridemia + Obesidad grado II. C
on antecedente de estancia hospitalaria en Terapia Respiratoria Intermedia (C7):
 del día 05.09.20 al 11.09.20. Con antecedente de infección pulmonar sobreagre
gada (En tratamiento con linezolid). Con PCR (+) Al pase de visita la paciente n
iega dolor torácico. Negó sintomatología urinaria o digestiva. La paciente ni
ega la presencia de disnea.



In [53]:
Lista_sintomas = [('Síntomas' + ':' + data['20']['nota']['Síntomas']).split()]

In [54]:
Pred_sin_nota20 = model.predict(Lista_sintomas)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
# Procesamos para ver las predicciones que está haciendo 

Palabra_nota_1 = []
for i in Lista_sintomas:
      Palabra_nota_1.append(i)

Predicciones_nota_1 = []
for i in Pred_sin_nota20:

      Predicciones_nota_1.append(i)

data_frame_sn20 = pd.DataFrame(list(zip(Palabra_nota_1, Predicciones_nota_1)), columns = ['Palabra de la nota 20 sintomas','Predicción de la nota 20 sintomas'])

In [56]:
Palabra_nota_1

[['Síntomas:',
  'Se',
  'trata',
  'Arellano',
  'Ojeda',
  'Marilú',
  'de',
  '52',
  'años',
  'de',
  'edad,',
  'con',
  'antece',
  'dentes',
  'de',
  'Diabetes',
  'Mellitus',
  'tipo',
  '2',
  '+',
  'Hipertrigliceridemia',
  '+',
  'Obesidad',
  'grado',
  'II.',
  'C',
  'on',
  'antecedente',
  'de',
  'estancia',
  'hospitalaria',
  'en',
  'Terapia',
  'Respiratoria',
  'Intermedia',
  '(C7):',
  'del',
  'día',
  '05.09.20',
  'al',
  '11.09.20.',
  'Con',
  'antecedente',
  'de',
  'infección',
  'pulmonar',
  'sobreagre',
  'gada',
  '(En',
  'tratamiento',
  'con',
  'linezolid).',
  'Con',
  'PCR',
  '(+)',
  'Al',
  'pase',
  'de',
  'visita',
  'la',
  'paciente',
  'n',
  'iega',
  'dolor',
  'torácico.',
  'Negó',
  'sintomatología',
  'urinaria',
  'o',
  'digestiva.',
  'La',
  'paciente',
  'ni',
  'ega',
  'la',
  'presencia',
  'de',
  'disnea.']]

In [57]:
li_1 = []
li_1 = list(zip(Palabra_nota_1[0], Predicciones_nota_1[0]))

In [58]:
li_1

[('Síntomas:', 'O'),
 ('Se', 'O'),
 ('trata', 'O'),
 ('Arellano', 'O'),
 ('Ojeda', 'O'),
 ('Marilú', 'O'),
 ('de', 'O'),
 ('52', 'O'),
 ('años', 'O'),
 ('de', 'O'),
 ('edad,', 'O'),
 ('con', 'O'),
 ('antece', 'O'),
 ('dentes', 'O'),
 ('de', 'O'),
 ('Diabetes', 'O'),
 ('Mellitus', 'O'),
 ('tipo', 'O'),
 ('2', 'O'),
 ('+', 'O'),
 ('Hipertrigliceridemia', 'B-SIN'),
 ('+', 'O'),
 ('Obesidad', 'O'),
 ('grado', 'O'),
 ('II.', 'O'),
 ('C', 'O'),
 ('on', 'O'),
 ('antecedente', 'O'),
 ('de', 'O'),
 ('estancia', 'O'),
 ('hospitalaria', 'O'),
 ('en', 'O'),
 ('Terapia', 'O'),
 ('Respiratoria', 'O'),
 ('Intermedia', 'O'),
 ('(C7):', 'O'),
 ('del', 'O'),
 ('día', 'O'),
 ('05.09.20', 'O'),
 ('al', 'O'),
 ('11.09.20.', 'O'),
 ('Con', 'O'),
 ('antecedente', 'O'),
 ('de', 'O'),
 ('infección', 'O'),
 ('pulmonar', 'O'),
 ('sobreagre', 'O'),
 ('gada', 'O'),
 ('(En', 'O'),
 ('tratamiento', 'O'),
 ('con', 'O'),
 ('linezolid).', 'O'),
 ('Con', 'O'),
 ('PCR', 'O'),
 ('(+)', 'O'),
 ('Al', 'O'),
 ('pase', 'O'),


In [59]:
Lista_oraciones_prediccionesn1 = []
contador1 = 0
for i in li_1:
  if 'B-SIN' == i[1] or 'I-SIN' == i[1]:
    Lista_oraciones_prediccionesn1.append([i, contador1])
  contador1 += 1


In [60]:
Lista_oraciones_prediccionesn1

[[('Hipertrigliceridemia', 'B-SIN'), 20],
 [('dolor', 'I-SIN'), 63],
 [('torácico.', 'I-SIN'), 64],
 [('disnea.', 'B-SIN'), 77]]